In [93]:
import pandas as pd
import geopandas as gpd

In [94]:
def conduct_reverse_geocoding(df: pd.DataFrame, gdf_shape):
    """
    enriches the dataframe with the federal states, counties and municipalities the long-lat data points are located in
    :param df: the germany dataframe with the long-lat data
    :param gdf_shape: shapefile with the federal state, county and municipality borders for Germany
    :return: df: original dataframe with additional country column
    """
    # convert df to GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.long, df.lat))

    # Merge DataFrames
    pointInPolys = gpd.sjoin(gdf, gdf_shape, how='left')

    # Drop columns we don't need
    # pointInPolys = pointInPolys.drop(
    #     columns=['french_shor', 'status', 'index_right', 'color_code', 'status'])
    return pointInPolys

In [95]:
# Read shapefiles as geodataframe
sf_federal_states = gpd.GeoDataFrame.from_file("./data/shapefiles/germany/federal_states/B-2020-AI001-2-5--AI0106--2023-01-03.shp")

sf_counties = gpd.GeoDataFrame.from_file("./data/shapefiles/germany/counties/K-2020-AI001-2-5--AI0106--2023-01-03.shp")

sf_municipalities = gpd.GeoDataFrame.from_file("./data/shapefiles/germany/municipalities/G-2020-AI001-2-5--AI0106--2023-01-03.shp")

# load germany data as df
df_germany = pd.read_csv("./data/final_data/germany_final.csv", sep=";")


In [96]:
df_germany.head()

,Unnamed: 0,avg_lat_ms,tests,devices,quarter,category,long,lat,country,avg_d_mbps,avg_u_mbps
0,0,43,4,2,2019-01-01,fixed,8.415527,55.024873,Germany,8,1
1,1,34,1,1,2019-01-01,fixed,8.421021,55.024873,Germany,33,9
2,2,29,1,1,2019-01-01,fixed,8.426514,55.024873,Germany,24,11
3,3,40,2,1,2019-01-01,fixed,8.432007,55.024873,Germany,35,7
4,4,33,4,3,2019-01-01,fixed,8.421021,55.021725,Germany,7,2


In [97]:
# convert CRS of shapefiles to fit CRS of CRS in Germany file (EPSG)
sf_federal_states = sf_federal_states.to_crs(4326)
sf_counties = sf_counties.to_crs(4326)
sf_municipalities = sf_municipalities.to_crs(4326)

In [98]:
# Perform geocoding
df_germany = conduct_reverse_geocoding(df_germany, sf_federal_states)
df_germany.drop(columns=["index_right"], inplace=True)

df_germany = conduct_reverse_geocoding(df_germany, sf_counties)
df_germany.drop(columns=["index_right"], inplace=True)

df_germany = conduct_reverse_geocoding(df_germany, sf_municipalities)


/var/folders/fx/lflfvn0n10963h3v1rgc6nvc0000gn/T/ipykernel_43481/1567522274.py:12: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  pointInPolys = gpd.sjoin(gdf, gdf_shape, how='left')
/var/folders/fx/lflfvn0n10963h3v1rgc6nvc0000gn/T/ipykernel_43481/1567522274.py:12: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  pointInPolys = gpd.sjoin(gdf, gdf_shape, how='left')
/var/folders/fx/lflfvn0n10963h3v1rgc6nvc0000gn/T/ipykernel_43481/1567522274.py:12: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right 

In [99]:
df_germany.drop(columns=
["id_left",         
"schluessel_left",
"jahr_left",
"ai0106_left",
"Shape_Leng_left",
"Shape_Area_left",
"id_right", 
"schluessel_right",
"id_right",
"jahr_right", 
"Shape_Leng_right",
"Shape_Area_right",
"index_right",
"id",
"jahr", 
"Shape_Leng",
"Shape_Area",
"ai0106_right",
"schluessel",
"ai0106"
], inplace=True)


In [100]:
# Skip columns that are not needed
df_germany.drop(columns=["Unnamed: 0", "long", "lat", "geometry"], inplace=True)

df_germany.rename(columns={"gen_left":"federal_state", "gen_right":"county", "gen":"municipality"}, inplace=True)

In [105]:
df_germany.sample(10)

,avg_lat_ms,tests,devices,quarter,category,country,avg_d_mbps,avg_u_mbps,federal_state,county,municipality
4131886,45,1,1,2020-01-01,mobile,Germany,28,29,Rheinland-Pfalz,Cochem-Zell,Ulmen
3339924,13,14,4,2022-04-01,fixed,Germany,38,21,Hessen,Groß-Gerau,Stockstadt am Rhein
2646987,31,4,4,2021-07-01,fixed,Germany,31,4,Sachsen,Chemnitz,Chemnitz
1600727,10,1,1,2020-07-01,fixed,Germany,2,9,Hessen,Gießen,Laubach
48912,10,1,1,2019-01-01,fixed,Germany,56,10,Nordrhein-Westfalen,Krefeld,Krefeld
716054,26,1,1,2019-10-01,fixed,Germany,2,3,Niedersachsen,Gifhorn,Groß Oesingen
508709,11,1,1,2019-07-01,fixed,Germany,36,10,Nordrhein-Westfalen,Dortmund,Dortmund
1565551,16,1,1,2020-07-01,fixed,Germany,1,7,Sachsen-Anhalt,Harz,Thale
3016200,15,7,2,2022-01-01,fixed,Germany,40,7,Nordrhein-Westfalen,Köln,Köln
4165923,29,2,2,2020-01-01,mobile,Germany,57,9,Sachsen,Chemnitz,Chemnitz


In [102]:
# save as csv
df_germany.to_csv("./data/final_data/germany_final.csv", sep=";")